In [1]:
import os
import glob
import numpy as np

def get_all_alpha_ry(label_root):
    labels = glob.glob(os.path.join(label_root, '*.txt'), recursive=True)
    Alphas = list()
    Rys = list()
    for i in range(len(labels)):
        with open(labels[i]) as f:
            lines = f.readlines()
            for idx, line in enumerate(lines):
                elements = line[:-1].split()
                if elements[0] == 'DontCare':
                    continue
                alpha = float(elements[3])
                ry = float(elements[14])
                Alphas.append(alpha)
                Rys.append(ry)
    Alphas = np.array(Alphas)
    Rys = np.array(Rys)
    return Alphas, Rys 

## GT part, run once and load npy in future

In [2]:
os.makedirs('orient_delta', exist_ok=True)
#gt_alphas, gt_rys = get_all_alpha_ry(label_root='./Kitti/training/label_2') # len 40570
#np.save('orient_delta/gt_alphas.npy', gt_alphas)
#np.save('orient_delta/gt_rys.npy', gt_rys)
gt_alphas = np.load('orient_delta/gt_alphas.npy')
gt_rys = np.load('orient_delta/gt_rys.npy')

## Eval part, for generated pred labels

In [3]:
# best sum() 40570, mean() 1
def angle_criterion(PRED, GT):
    cos_delta = np.cos(PRED - GT)
    return cos_delta
    #return (1+cos_delta)/2

In [9]:
eval_folder = 'BL_class2angle'
os.makedirs(f'orient_delta/{eval_folder}', exist_ok=True)
#eval_alphas, eval_rys = get_all_alpha_ry(label_root=eval_folder) # len 40570
#np.save(f'orient_delta/{eval_folder}/eval_alphas.npy', eval_alphas)
#np.save(f'orient_delta/{eval_folder}/eval_rys.npy', eval_rys)
eval_alphas = np.load(f'orient_delta/{eval_folder}/eval_alphas.npy')
eval_rys = np.load(f'orient_delta/{eval_folder}/eval_rys.npy')
delta_alpha = angle_criterion(eval_alphas, gt_alphas)
delta_ry = angle_criterion(eval_rys, gt_rys)
print(f'Alpha sum:{delta_alpha.sum()}, mean:{delta_alpha.mean()}')
print(f'Ry sum:{delta_ry.sum()}, mean:{delta_ry.mean()}')

Alpha sum:40061.986447780764, mean:0.987478098293832
Ry sum:39995.551652176895, mean:0.985840563277715


# 24 BIN

In [5]:
eval_folder = 'BL_24bin'
os.makedirs(f'orient_delta/{eval_folder}', exist_ok=True)
#eval_alphas, eval_rys = get_all_alpha_ry(label_root=eval_folder) # len 40570
#np.save(f'orient_delta/{eval_folder}/eval_alphas.npy', eval_alphas)
#np.save(f'orient_delta/{eval_folder}/eval_rys.npy', eval_rys)
eval_alphas = np.load(f'orient_delta/{eval_folder}/eval_alphas.npy')
eval_rys = np.load(f'orient_delta/{eval_folder}/eval_rys.npy')
delta_alpha = angle_criterion(eval_alphas, gt_alphas)
delta_ry = angle_criterion(eval_rys, gt_rys)
print(f'Alpha sum:{delta_alpha.sum()}, mean:{delta_alpha.mean()}')
print(f'Ry sum:{delta_ry.sum()}, mean:{delta_ry.mean()}')

Alpha sum:40044.89146299424, mean:0.9870567281980339
Ry sum:39994.49876609354, mean:0.9858146109463529


In [21]:
eval_folder = 'BL_24bin_no_round'
os.makedirs(f'orient_delta/{eval_folder}', exist_ok=True)
eval_alphas, eval_rys = get_all_alpha_ry(label_root=eval_folder) # len 40570
np.save(f'orient_delta/{eval_folder}/eval_alphas.npy', eval_alphas)
np.save(f'orient_delta/{eval_folder}/eval_rys.npy', eval_rys)
#eval_alphas = np.load(f'orient_delta/{eval_folder}/eval_alphas.npy')
#eval_rys = np.load(f'orient_delta/{eval_folder}/eval_rys.npy')
delta_alpha = angle_criterion(eval_alphas, gt_alphas)
delta_ry = angle_criterion(eval_rys, gt_rys)
print(f'Alpha sum:{delta_alpha.sum()}, mean:{delta_alpha.mean()}')
print(f'Ry sum:{delta_ry.sum()}, mean:{delta_ry.mean()}')

Alpha sum:40044.89146299424, mean:0.9870567281980339
Ry sum:39994.49876609354, mean:0.9858146109463529


In [16]:
eval_folder = 'Group_24bin'
os.makedirs(f'orient_delta/{eval_folder}', exist_ok=True)
eval_alphas, eval_rys = get_all_alpha_ry(label_root=eval_folder) # len 40570
np.save(f'orient_delta/{eval_folder}/eval_alphas.npy', eval_alphas)
np.save(f'orient_delta/{eval_folder}/eval_rys.npy', eval_rys)
#eval_alphas1 = np.load(f'orient_delta/{eval_folder}/eval_alphas.npy')
#eval_rys1 = np.load(f'orient_delta/{eval_folder}/eval_rys.npy')
delta_alpha = angle_criterion(eval_alphas, gt_alphas)
delta_ry = angle_criterion(eval_rys, gt_rys)
print(f'Alpha sum:{delta_alpha.sum()}, mean:{delta_alpha.mean()}')
print(f'Ry sum:{delta_ry.sum()}, mean:{delta_ry.mean()}')

Alpha sum:39913.558617678835, mean:0.9838195370391628
Ry sum:39870.29000176671, mean:0.9827530195160639


In [20]:
eval_folder = 'Group_24bin_no_round'
os.makedirs(f'orient_delta/{eval_folder}', exist_ok=True)
eval_alphas, eval_rys = get_all_alpha_ry(label_root=eval_folder) # len 40570
np.save(f'orient_delta/{eval_folder}/eval_alphas.npy', eval_alphas)
np.save(f'orient_delta/{eval_folder}/eval_rys.npy', eval_rys)
#eval_alphas2 = np.load(f'orient_delta/{eval_folder}/eval_alphas.npy')
#eval_rys2 = np.load(f'orient_delta/{eval_folder}/eval_rys.npy')
delta_alpha = angle_criterion(eval_alphas, gt_alphas)
delta_ry = angle_criterion(eval_rys, gt_rys)
print(f'Alpha sum:{delta_alpha.sum()}, mean:{delta_alpha.mean()}')
print(f'Ry sum:{delta_ry.sum()}, mean:{delta_ry.mean()}')

Alpha sum:39976.654125292625, mean:0.9853747627629437
Ry sum:39929.92543408659, mean:0.9842229586908205


In [18]:
(eval_alphas1 == eval_alphas1).all()

True

# 36 BIN

In [6]:
eval_folder = 'BL_36bin'
os.makedirs(f'orient_delta/{eval_folder}', exist_ok=True)
#eval_alphas, eval_rys = get_all_alpha_ry(label_root=eval_folder) # len 40570
#np.save(f'orient_delta/{eval_folder}/eval_alphas.npy', eval_alphas)
#np.save(f'orient_delta/{eval_folder}/eval_rys.npy', eval_rys)
eval_alphas = np.load(f'orient_delta/{eval_folder}/eval_alphas.npy')
eval_rys = np.load(f'orient_delta/{eval_folder}/eval_rys.npy')
delta_alpha = angle_criterion(eval_alphas, gt_alphas)
delta_ry = angle_criterion(eval_rys, gt_rys)
print(f'Alpha sum:{delta_alpha.sum()}, mean:{delta_alpha.mean()}')
print(f'Ry sum:{delta_ry.sum()}, mean:{delta_ry.mean()}')

Alpha sum:39976.52968460096, mean:0.9853716954547932
Ry sum:39929.7178208485, mean:0.9842178412829307


In [19]:
eval_folder = 'BL_36bin_no_round'
os.makedirs(f'orient_delta/{eval_folder}', exist_ok=True)
eval_alphas, eval_rys = get_all_alpha_ry(label_root=eval_folder) # len 40570
np.save(f'orient_delta/{eval_folder}/eval_alphas.npy', eval_alphas)
np.save(f'orient_delta/{eval_folder}/eval_rys.npy', eval_rys)
#eval_alphas = np.load(f'orient_delta/{eval_folder}/eval_alphas.npy')
#eval_rys = np.load(f'orient_delta/{eval_folder}/eval_rys.npy')
delta_alpha = angle_criterion(eval_alphas, gt_alphas)
delta_ry = angle_criterion(eval_rys, gt_rys)
print(f'Alpha sum:{delta_alpha.sum()}, mean:{delta_alpha.mean()}')
print(f'Ry sum:{delta_ry.sum()}, mean:{delta_ry.mean()}')

Alpha sum:39976.654125292625, mean:0.9853747627629437
Ry sum:39929.92543408659, mean:0.9842229586908205


In [8]:
eval_folder = 'Group_36bin'
os.makedirs(f'orient_delta/{eval_folder}', exist_ok=True)
eval_alphas, eval_rys = get_all_alpha_ry(label_root=eval_folder) # len 40570
np.save(f'orient_delta/{eval_folder}/eval_alphas.npy', eval_alphas)
np.save(f'orient_delta/{eval_folder}/eval_rys.npy', eval_rys)
#eval_alphas = np.load(f'orient_delta/{eval_folder}/eval_alphas.npy')
#eval_rys = np.load(f'orient_delta/{eval_folder}/eval_rys.npy')
delta_alpha = angle_criterion(eval_alphas, gt_alphas)
delta_ry = angle_criterion(eval_rys, gt_rys)
print(f'Alpha sum:{delta_alpha.sum()}, mean:{delta_alpha.mean()}')
print(f'Ry sum:{delta_ry.sum()}, mean:{delta_ry.mean()}')

Alpha sum:39913.558617678835, mean:0.9838195370391628
Ry sum:39870.29000176671, mean:0.9827530195160639


In [22]:
eval_folder = 'Group_36bin_no_round'
os.makedirs(f'orient_delta/{eval_folder}', exist_ok=True)
eval_alphas, eval_rys = get_all_alpha_ry(label_root=eval_folder) # len 40570
np.save(f'orient_delta/{eval_folder}/eval_alphas.npy', eval_alphas)
np.save(f'orient_delta/{eval_folder}/eval_rys.npy', eval_rys)
#eval_alphas = np.load(f'orient_delta/{eval_folder}/eval_alphas.npy')
#eval_rys = np.load(f'orient_delta/{eval_folder}/eval_rys.npy')
delta_alpha = angle_criterion(eval_alphas, gt_alphas)
delta_ry = angle_criterion(eval_rys, gt_rys)
print(f'Alpha sum:{delta_alpha.sum()}, mean:{delta_alpha.mean()}')
print(f'Ry sum:{delta_ry.sum()}, mean:{delta_ry.mean()}')

Alpha sum:39913.558617678835, mean:0.9838195370391628
Ry sum:39870.29000176671, mean:0.9827530195160639
